In [5]:
import pandas as pd
import numpy as np
file_path1='./doctors_data_multireward.csv'
# data = pd.read_csv(file_path)
data1=pd.read_csv(file_path1)
data1.head(), data1.columns

(        Doctor Speciality         Doctor Name  Ratings  Experience  \
 0  Allergy and Immunology      Dr. James Mays     1.85        5.52   
 1  Allergy and Immunology  Dr. Paula Castillo     1.80        4.32   
 2  Allergy and Immunology  Dr. Kristin Turner     2.27       16.94   
 3  Allergy and Immunology  Dr. April Mcintosh     4.21        3.04   
 4  Allergy and Immunology  Dr. Courtney Price     2.59       22.20   
 
    Distance from Patient  Availability  Cost of Services  
 0                   5.45             0              8.31  
 1                  13.66             1              5.38  
 2                  10.99             1             21.30  
 3                   0.00             1              3.59  
 4                  15.67             1              6.50  ,
 Index(['Doctor Speciality', 'Doctor Name', 'Ratings', 'Experience',
        'Distance from Patient', 'Availability', 'Cost of Services'],
       dtype='object'))

In [6]:
cols_drop = ["Availability","Distance from Patient","Cost of Services"]
data1.drop(columns=cols_drop, inplace=True)
# import matplotlib.pyplot as plt


In [7]:
data1.columns

Index(['Doctor Speciality', 'Doctor Name', 'Ratings', 'Experience'], dtype='object')

In [8]:
class MultiArmedBanditExperience:
    def __init__(self, data, epsilon=0.5, ratings_weight=0.0, experience_weight=0.0):
        self.data = data
        self.epsilon = epsilon
        self.ratings_weight = ratings_weight
        self.experience_weight = experience_weight
        
        # Normalize Ratings and Experience
        self.data['Normalized Ratings'] = (self.data['Ratings'] - self.data['Ratings'].min()) / (self.data['Ratings'].max() - self.data['Ratings'].min())
        self.data['Normalized Experience'] = (self.data['Experience'] - self.data['Experience'].min()) / (self.data['Experience'].max() - self.data['Experience'].min())
        
        # Compute composite scores
        self.data['Composite Score'] = self.data['Normalized Ratings'] * self.ratings_weight + self.data['Normalized Experience'] * self.experience_weight

        # Initialize dictionaries to keep track of composite scores and counts for each doctor
        self.scores = self.data.groupby(['Doctor Speciality', 'Doctor Name'])['Composite Score'].mean().to_dict()
        self.counts = self.data.groupby(['Doctor Speciality', 'Doctor Name'])['Composite Score'].count().to_dict()

    def recommend_doctor(self, specialists):
        recommendations = {}
        for specialist in specialists:
            doctors = self.data[self.data['Doctor Speciality'] == specialist]
            if doctors.empty:
                recommendations[specialist] = "No doctors available for this specialty."
                continue

            # Sort doctors by composite score in descending order
            sorted_doctors = doctors.sort_values(by='Composite Score', ascending=False).head(5)
            top_doctors = []
            recommended_names = set()  # Set to store recommended doctor names

            for _ in range(5):  # Select top 5 doctors based on stochastic policy
                if np.random.rand() < self.epsilon:  # Explore
                    # Randomly choose a doctor
                    recommendation = doctors.sample()
                else:  # Exploit
                    max_score = -np.inf
                    for index, row in doctors.iterrows():
                        key = (specialist, row['Doctor Name'])
                        if self.scores[key] > max_score and row['Doctor Name'] not in recommended_names:
                            max_score = self.scores[key]
                            recommendation = row.to_frame().T
                            recommended_names.add(row['Doctor Name'])  # Add doctor name to set of recommended names

                doctor_name = recommendation['Doctor Name'].iloc[0]
                doctor_rating = recommendation['Ratings'].iloc[0]
                doctor_experience = recommendation['Experience'].iloc[0]
                top_doctors.append((doctor_name, doctor_rating, doctor_experience))

            recommendations[specialist] = top_doctors

        return recommendations


# Usage Example:
specialists_input = input("Enter the list of specialists you are looking for, separated by commas: ").split(',')
user_exp_weight = float(input("On a scale of 0-9, how much weight do you want to give to the doctor's experience: ")) / 10
ratings_weight = 1 - user_exp_weight


bandit_with_experience = MultiArmedBanditExperience(data1, ratings_weight=ratings_weight, experience_weight=user_exp_weight)

results = bandit_with_experience.recommend_doctor(specialists_input)
print(f"Recommended Doctors with Ratings and Experience for : {specialists_input}")
for specialist, top_doctors in results.items():
   
    print("-" * 60)
    print(f"Specilaity : {specialist}")
    print("-" * 60)
    print(f"{'Doctor Name':30s} | {'Rating':10s} | {'Experience':10s}")
    print("-" * 60)
    for doctor in top_doctors:
        print(f"{doctor[0]:30s} | {doctor[1]:10.2f} | {doctor[2]:10.2f}")
print("-" * 60)

Recommended Doctors with Ratings and Experience for : ['Cardiology']
------------------------------------------------------------
Specilaity : Cardiology
------------------------------------------------------------
Doctor Name                    | Rating     | Experience
------------------------------------------------------------
Dr. Samuel Griffith            |       4.97 |     130.31
Dr. Sharon Guerrero            |       3.45 |       0.13
Dr. Shannon Henderson          |       6.83 |      86.04
Dr. David Stevenson            |       6.09 |      68.22
Dr. Donna Francis              |       2.59 |      10.76
------------------------------------------------------------


In [9]:
import joblib
from bandit_experience import MultiArmedBanditExperience

# Assume data1 is already defined
user_exp_weight = 0.7  # Example weight
ratings_weight = 1 - user_exp_weight

bandit_with_experience = MultiArmedBanditExperience(data1, ratings_weight=ratings_weight, experience_weight=user_exp_weight)
joblib.dump(bandit_with_experience, 'bandit.joblib')


['bandit.joblib']